In [1]:
import time
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS 
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower,desc, mean, explode, lit
# from pyspark.sql import *
import json
import requests
import pandas as pd

In [2]:
#Creamos la sesión y cargamos los distintos csv que tenemos
spark = SparkSession.builder.master("local[2]").getOrCreate()
ratings = spark.read.csv('/data/animeratings/rating_complete.csv', header=True, inferSchema=True, encoding='utf-8')

#Es importante establecer en esta tabla el header=False, porque esta tabla no tiene encabezados y si lo ponemos a false pone headers automáticos
anadir = spark.read.csv('/data/animeratings/valoraciones_EP.csv', header=False, inferSchema=True, encoding='utf-8')

anime = spark.read.csv('/data/animeratings/anime.csv', header=True, inferSchema=True, encoding='utf-8')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#Renombramos las cabeceras para que coincidan en ambas tablas y las unimos
cambio = anadir.withColumnRenamed('_c0','user_id').withColumnRenamed('_c1','anime_id').withColumnRenamed('_c2','rating')
add2 = ratings.union(cambio)
ult = add2.withColumnRenamed('rating','ratings')
todo = anime.join(ult, ult.anime_id  == anime.ID)

<h1>1. ANÁLISIS EXPLORATORIO</h1>

In [5]:
#Mostramos los 5 géneros mejor y menor valorados para hacer una comparación de cuales gustan más
genero = todo.groupBy("Genres").mean("ratings")
renomG = genero.withColumnRenamed("avg(ratings)", "ratings")
generoF = renomG.filter((renomG.Genres != "Unknown"))
generoF.sort(desc("avg(ratings)")).limit(5).show(truncate=False)
generoF.orderBy(['avg(ratings)'], ascending = [True]).limit(5).show(truncate=False)

+-------------------------------------------------------------------+-----------------+
|Genres                                                             |ratings          |
+-------------------------------------------------------------------+-----------------+
|Adventure, Demons, Fantasy, Historical, Kids, Magic, Supernatural  |10.0             |
|Action, Military, Adventure, Comedy, Drama, Magic, Fantasy, Shounen|9.237009769219878|
|Thriller, Sci-Fi                                                   |9.18422767406249 |
|Action, Sci-Fi, Comedy, Historical, Parody, Drama, Samurai, Shounen|9.146341463414634|
|Military, Sci-Fi, Space, Drama                                     |9.111402623612513|
+-------------------------------------------------------------------+-----------------+



+--------------------------------------------+------------------+
|Genres                                      |ratings           |
+--------------------------------------------+------------------+
|Sci-Fi, Horror                              |2.1830747964982335|
|Action, Kids, Adventure, Super Power, Demons|2.388888888888889 |
|Kids, Adventure, Space                      |2.7674418604651163|
|Comedy, Historical, Fantasy                 |2.875             |
|Action, Super Power, Demons, Seinen         |2.9223803363518757|
+--------------------------------------------+------------------+



In [6]:
#Agrupamos las productoras y calculamos la media de su rating junto con el numero de produciones que tiene cada una
infoR = todo.groupBy("Producers").mean("ratings")
#Agrupamos y contamos las apariciones, que serian la cantidad de series producidas
infoN = todo.groupBy("Producers").count() 
renameA = infoR.withColumnRenamed("avg(ratings)", "rating_medio") 
renameB = infoN.withColumnRenamed("count", "nº_produciones")

producciones = renameA.join(renameB, on='Producers')

#Filtramos para limpiar de datos que no necesitamos el dataframe y ordenamos
mejoresProductores = producciones.filter((producciones.Producers != "Unknown")).distinct()
peoresProductores = producciones.filter((producciones.Producers != "Unknown")).distinct()

In [7]:
mejoresProductores.sort(desc("rating_medio")).limit(5).show(truncate=False)
peoresProductores.orderBy(['rating_medio'], ascending = [True]).limit(5).show(truncate=False)

+-----------------------------------------------------------------------------+-----------------+--------------+
|Producers                                                                    |rating_medio     |nº_produciones|
+-----------------------------------------------------------------------------+-----------------+--------------+
|TV Tokyo, Aniplex, Dentsu                                                    |9.24564778301433 |20794         |
|Aniplex, Square Enix, Mainichi Broadcasting System, Studio Moriken           |9.237009769219878|134197        |
|Kitty Films, K-Factory                                                       |9.224729815638906|9438          |
|TV Tokyo, Aniplex, Dentsu, Trinity Sound, Miracle Robo, Studio Jack, Shueisha|9.150585006500073|27692         |
|TV Tokyo, Aniplex, Dentsu, Trinity Sound, Audio Highs, Miracle Robo, Shueisha|9.110528462390025|34326         |
+-----------------------------------------------------------------------------+-----------------

+------------------------------------------+------------------+--------------+
|Producers                                 |rating_medio      |nº_produciones|
+------------------------------------------+------------------+--------------+
|Tsuburaya Productions, NHK Enterprises    |2.0               |2             |
|Idea Factory                              |2.130665920537213 |7148          |
|Idea Factory, King Records, Design Factory|2.1830747964982335|13022         |
|UM Productions                            |2.234234234234234 |222           |
|Showgate, Tokyo MX                        |2.7772511848341233|211           |
+------------------------------------------+------------------+--------------+



<h1> EJERCICIO 2</h1>

In [8]:
#Creo el conjunto de datos de peliculas
movie = todo.filter(todo.Type == "Movie").select("user_id","anime_id","Name","English name","ratings","Type")

(movie_training, movie_test) = movie.randomSplit([0.8, 0.2])
alsMo = ALS(maxIter=10, regParam=0.16, userCol="user_id", itemCol="anime_id", ratingCol="ratings", coldStartStrategy="drop")
modelMo = alsMo.fit(movie_training)
predictionsMo = modelMo.transform(movie_test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="ratings", predictionCol="prediction")
rmse = evaluator.evaluate(predictionsMo) 
print("Root-mean-square error = " + str(rmse))

22/12/22 08:52:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/22 08:52:22 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/12/22 08:52:23 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Root-mean-square error = 1.1626111695644032


In [9]:
#Hacemos la recomendación de 5 peliculas para el usuario 666666
userMo = movie.where(movie.user_id == 666666)
recomendacionMo = modelMo.recommendForUserSubset(userMo, 5)

/home/21609154Carlos/.local/lib/python3.10/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [27]:
recomendacionMo.show(truncate=False)
recomendacionMo.printSchema()

+-------+-----------------------------------------------------------------------------------------+
|user_id|recommendations                                                                          |
+-------+-----------------------------------------------------------------------------------------+
|666666 |[{28023, 8.705702}, {199, 8.597341}, {32281, 8.406871}, {164, 8.397985}, {431, 8.388902}]|
+-------+-----------------------------------------------------------------------------------------+

root
 |-- user_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- anime_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [10]:
#Uso explode para dividir el struct creado al realizar las recomendaciones y con select elegimos las columnas del struct
reconMo = recomendacionMo.withColumn("recomendaciones",explode("recommendations")).drop("recommendations")
reconMo2 = reconMo.select(col("user_id"),col("recomendaciones.anime_id"),col("recomendaciones.rating"))
j = reconMo2.show()

+-------+--------+--------+
|user_id|anime_id|  rating|
+-------+--------+--------+
| 666666|   28023|8.705702|
| 666666|     199|8.597341|
| 666666|   32281|8.406871|
| 666666|     164|8.397985|
| 666666|     431|8.388902|
+-------+--------+--------+



In [11]:
#Primero modificamos el dataframe para elminar las columnas que no necesitamos y se repiten
#Mediante un join, insertamos los nombres de los anime en la tabla donde tenemos las predicciones
movieMod = movie.drop("user_id", "ratings").distinct().sort(desc("anime_id"))
finalMo = reconMo2.join(movieMod, on = 'anime_id').distinct()
meMo = finalMo.show()

+--------+-------+--------+--------------------+--------------------+-----+
|anime_id|user_id|  rating|                Name|        English name| Type|
+--------+-------+--------+--------------------+--------------------+-----+
|   32281| 666666|8.406871|      Kimi no Na wa.|          Your Name.|Movie|
|     431| 666666|8.388902| Howl no Ugoku Shiro|Howl's Moving Castle|Movie|
|     164| 666666|8.397985|       Mononoke Hime|   Princess Mononoke|Movie|
|     199| 666666|8.597341|Sen to Chihiro no...|       Spirited Away|Movie|
|   28023| 666666|8.705702|   Tsuru no Sugomori|             Unknown|Movie|
+--------+-------+--------+--------------------+--------------------+-----+



In [12]:
#Creo el conjunto de datos de peliculas
tv = todo.filter(todo.Type == "TV").select("user_id","anime_id","Name","English name","ratings","Type")

(tv_training, tv_test) = tv.randomSplit([0.8, 0.2])
alsTv = ALS(maxIter=10, regParam=0.16, userCol="user_id", itemCol="anime_id", ratingCol="ratings", coldStartStrategy="drop")
modelTv = alsTv.fit(tv_training)
predictionsTv = modelTv.transform(tv_test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="ratings", predictionCol="prediction")
rmse = evaluator.evaluate(predictionsTv) 
print("Root-mean-square error = " + str(rmse))

22/12/22 08:54:52 WARN MemoryStore: Not enough space to cache rdd_523_7 in memory! (computed 60.7 MiB so far)
22/12/22 08:54:52 WARN BlockManager: Persisting block rdd_523_7 to disk instead.


Root-mean-square error = 1.143857978443533


In [13]:
#Hacemos la recomendación de 5 series para el usuario 666666
userTv = tv.where(tv.user_id == 666666)
recomendacionTv = modelTv.recommendForUserSubset(userTv, 5)

In [14]:
#Uso explode para dividir el struct creado al realizar las recomendaciones y con select elegimos las columnas del struct
reconTv = recomendacionTv.withColumn("recomendaciones",explode("recommendations")).drop("recommendations")
reconTv2 = reconTv.select(col("user_id"),col("recomendaciones.anime_id"),col("recomendaciones.rating"))
j = reconTv2.show()

+-------+--------+---------+
|user_id|anime_id|   rating|
+-------+--------+---------+
| 666666|    2904|  8.80052|
| 666666|    9253| 8.749858|
| 666666|    4181| 8.723085|
| 666666|   38524|8.6793375|
| 666666|   28977| 8.675263|
+-------+--------+---------+



In [15]:
#Primero modificamos el dataframe para elminar las columnas que no necesitamos y se repiten
#Mediante un join, insertamos los nombres de los anime en la tabla donde tenemos las predicciones
tvMod = tv.drop("user_id", "ratings").distinct().sort(desc("anime_id"))
finalTv = reconTv2.join(tvMod, on = 'anime_id').distinct()
meTv = finalTv.show()

+--------+-------+---------+--------------------+--------------------+----+
|anime_id|user_id|   rating|                Name|        English name|Type|
+--------+-------+---------+--------------------+--------------------+----+
|    2904| 666666|  8.80052|Code Geass: Hangy...|Code Geass:Lelouc...|  TV|
|    9253| 666666| 8.749858|         Steins;Gate|         Steins;Gate|  TV|
|    4181| 666666| 8.723085|Clannad: After Story|Clannad ~After St...|  TV|
|   38524| 666666|8.6793375|Shingeki no Kyoji...|Attack on Titan S...|  TV|
|   28977| 666666| 8.675263|            Gintama°|    Gintama Season 4|  TV|
+--------+-------+---------+--------------------+--------------------+----+



In [16]:
unir = finalTv.union(finalMo)
orden = unir.sort(desc("rating")).show(truncate=False)


+--------+-------+---------+----------------------------------+--------------------------------------+-----+
|anime_id|user_id|rating   |Name                              |English name                          |Type |
+--------+-------+---------+----------------------------------+--------------------------------------+-----+
|2904    |666666 |8.80052  |Code Geass: Hangyaku no Lelouch R2|Code Geass:Lelouch of the Rebellion R2|TV   |
|9253    |666666 |8.749858 |Steins;Gate                       |Steins;Gate                           |TV   |
|4181    |666666 |8.723085 |Clannad: After Story              |Clannad ~After Story~                 |TV   |
|28023   |666666 |8.705702 |Tsuru no Sugomori                 |Unknown                               |Movie|
|38524   |666666 |8.6793375|Shingeki no Kyojin Season 3 Part 2|Attack on Titan Season 3 Part 2       |TV   |
|28977   |666666 |8.675263 |Gintama°                          |Gintama Season 4                      |TV   |
|199     |666666 |8

<h1>EJERCICIO 3</h1>

In [17]:
#Creamos los arrays con los anime_id de las listas de Tv y Movie que usaremos más adelante
listTv = finalTv.rdd.map(lambda x: x.anime_id).collect()
listMo = finalMo.rdd.map(lambda x: x.anime_id).collect()
print(listTv)
print(listMo)

[2904, 9253, 4181, 38524, 28977]
[28023, 199, 32281, 164, 431]


In [18]:
#Creamos el dataframe vacio donde volcaremos el json
dataFinalTv = pd.DataFrame()

#Recorremos el array de la lista anterior y hacemos un append para realizar la union del json a la tabla vacia
for x in listTv:
    url = "https://api.jikan.moe/v4/anime/" + str(x) + "/full"
    response = requests.get(url).text
    data = json.loads(response)
    df = pd.json_normalize(data)
    dataFinalTv = dataFinalTv.append(df, ignore_index=True)

/tmp/ipykernel_2903833/2478066977.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFinalTv = dataFinalTv.append(df, ignore_index=True)
/tmp/ipykernel_2903833/2478066977.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFinalTv = dataFinalTv.append(df, ignore_index=True)
/tmp/ipykernel_2903833/2478066977.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFinalTv = dataFinalTv.append(df, ignore_index=True)
/tmp/ipykernel_2903833/2478066977.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFinalTv = dataFinalTv.append(df, ignore_index=True)
/tmp/ipykernel_2903833/2478066977.py:10: FutureWarning: The frame.append

In [19]:
#Mostramos la información extra     
mostrarTv=dataFinalTv[["data.mal_id","data.episodes","data.title","data.year","data.synopsis"]]
mostrarTv.style.hide_index()

/tmp/ipykernel_2903833/819404743.py:3: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  mostrarTv.style.hide_index()


data.mal_id,data.episodes,data.title,data.year,data.synopsis
2904,25,Code Geass: Hangyaku no Lelouch R2,2008,"One year has passed since the Black Rebellion, a failed uprising against the Holy Britannian Empire led by the masked vigilante Zero, who is now missing. At a loss without their revolutionary leader, Area 11's resistance group—the Black Knights—find themselves too powerless to combat the brutality inflicted upon the Elevens by Britannia, which has increased significantly in order to crush any hope of a future revolt. Lelouch Lamperouge, having lost all memory of his double life, is living peacefully alongside his friends as a high school student at Ashford Academy. His former partner C.C., unable to accept this turn of events, takes it upon herself to remind him of his past purpose, hoping that the mastermind Zero will rise once again to finish what he started, in this thrilling conclusion to the series. [Written by MAL Rewrite]"
9253,24,Steins;Gate,2011,"Eccentric scientist Rintarou Okabe has a never-ending thirst for scientific exploration. Together with his ditzy but well-meaning friend Mayuri Shiina and his roommate Itaru Hashida, Rintarou founds the Future Gadget Laboratory in the hopes of creating technological innovations that baffle the human psyche. Despite claims of grandeur, the only notable ""gadget"" the trio have created is a microwave that has the mystifying power to turn bananas into green goo. However, when Rintarou decides to attend neuroscientist Kurisu Makise's conference on time travel, he experiences a series of strange events that lead him to believe that there is more to the ""Phone Microwave"" gadget than meets the eye. Apparently able to send text messages into the past using the microwave, Rintarou dabbles further with the ""time machine,"" attracting the ire and attention of the mysterious organization SERN. Due to the novel discovery, Rintarou and his friends find themselves in an ever-present danger. As he works to mitigate the damage his invention has caused to the timeline, he is not only fighting a battle to save his loved ones, but also one against his degrading sanity. [Written by MAL Rewrite]"
4181,24,Clannad: After Story,2008,"Clannad: After Story, the sequel to the critically acclaimed slice-of-life series Clannad, begins after Tomoya Okazaki and Nagisa Furukawa graduate from high school. Together, they experience the emotional rollercoaster of growing up. Unable to decide on a course for his future, Tomoya learns the value of a strong work ethic and discovers the strength of Nagisa's support. Through the couple's dedication and unity of purpose, they push forward to confront their personal problems, deepen their old relationships, and create new bonds. Time also moves on in the Illusionary World. As the plains grow cold with the approach of winter, the Illusionary Girl and the Garbage Doll are presented with a difficult situation that reveals the World's true purpose. Based on the visual novel by Key and produced by Kyoto Animation, Clannad: After Story is an impactful drama highlighting the importance of family and the struggles of adulthood. [Written by MAL Rewrite]"
38524,10,Shingeki no Kyojin Season 3 Part 2,2019,"Seeking to restore humanity's diminishing hope, the Survey Corps embark on a mission to retake Wall Maria, where the battle against the merciless ""Titans"" takes the stage once again. Returning to the tattered Shiganshina District that was once his home, Eren Yeager and the Corps find the town oddly unoccupied by Titans. Even after the outer gate is plugged, they strangely encounter no opposition. The mission progresses smoothly until Armin Arlert, highly suspicious of the enemy's absence, discovers distressing signs of a potential scheme against them. Shingeki no Kyojin Season 3 Part 2 follows Eren as he vows to take back everything that was once his. Alongside him, the Survey Corps strive—through countless sacrifices—to carve a path towards victory and uncover the

In [28]:
#Creamos el dataframe vacio donde volcaremos el json
dataFinalMo = pd.DataFrame()

#Recorremos el array de la lista anterior y hacemos un append para realizar la union del json a la tabla vacia
for x in listMo:
    url = "https://api.jikan.moe/v4/anime/" + str(x) + "/full"
    response = requests.get(url).text
    data = json.loads(response)
    df = pd.json_normalize(data)
    dataFinalMo = dataFinalMo.append(df, ignore_index=True)

/tmp/ipykernel_2903833/706812944.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFinalMo = dataFinalMo.append(df, ignore_index=True)
/tmp/ipykernel_2903833/706812944.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFinalMo = dataFinalMo.append(df, ignore_index=True)
/tmp/ipykernel_2903833/706812944.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFinalMo = dataFinalMo.append(df, ignore_index=True)
/tmp/ipykernel_2903833/706812944.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataFinalMo = dataFinalMo.append(df, ignore_index=True)
/tmp/ipykernel_2903833/706812944.py:10: FutureWarning: The frame.append meth

In [29]:
#Mostramos la información extra de las peliculas
mostrarMo=dataFinalMo[["data.mal_id","data.duration","data.title","data.rating","data.synopsis"]]
mostrarMo.style.hide_index()

/tmp/ipykernel_2903833/363655212.py:3: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  mostrarMo.style.hide_index()


data.mal_id,data.duration,data.title,data.rating,data.synopsis
28023,17 min,Tsuru no Sugomori,G - All Ages,"An earnest tale of animal cooperation, as two out-of-place chickens try to make their home on an island inhabited by cranes. When they are attacked by an eagle, their adoptive parent cranes are killed, but the chickens organize a counterstrike. Eventually, peace is restored to the island. Based on a story by Teru Takakura, this anime was made over a three-year period by around 80 members of the Japan Film Broadcasting Industry Association under the pseudonymous lead of Shibayama. (Source: The Anime Encyclopedia)"
199,2 hr 4 min,Sen to Chihiro no Kamikakushi,PG - Children,"Stubborn, spoiled, and naïve, 10-year-old Chihiro Ogino is less than pleased when she and her parents discover an abandoned amusement park on the way to their new house. Cautiously venturing inside, she realizes that there is more to this place than meets the eye, as strange things begin to happen once dusk falls. Ghostly apparitions and food that turns her parents into pigs are just the start—Chihiro has unwittingly crossed over into the spirit world. Now trapped, she must summon the courage to live and work amongst spirits, with the help of the enigmatic Haku and the cast of unique characters she meets along the way. Vivid and intriguing, Sen to Chihiro no Kamikakushi tells the story of Chihiro's journey through an unfamiliar world as she strives to save her parents and return home. [Written by MAL Rewrite]"
32281,1 hr 46 min,Kimi no Na wa.,PG-13 - Teens 13 or older,"Mitsuha Miyamizu, a high school girl, yearns to live the life of a boy in the bustling city of Tokyo—a dream that stands in stark contrast to her present life in the countryside. Meanwhile in the city, Taki Tachibana lives a busy life as a high school student while juggling his part-time job and hopes for a future in architecture. One day, Mitsuha awakens in a room that is not her own and suddenly finds herself living the dream life in Tokyo—but in Taki's body! Elsewhere, Taki finds himself living Mitsuha's life in the humble countryside. In pursuit of an answer to this strange phenomenon, they begin to search for one another. Kimi no Na wa. revolves around Mitsuha and Taki's actions, which begin to have a dramatic impact on each other's lives, weaving them into a fabric held together by fate and circumstance. [Written by MAL Rewrite]"
164,2 hr 13 min,Mononoke Hime,PG-13 - Teens 13 or older,"When an Emishi village is attacked by a fierce demon boar, the young prince Ashitaka puts his life at stake to defend his tribe. With its dying breath, the beast curses the prince's arm, granting him demonic powers while gradually siphoning his life away. Instructed by the village elders to travel westward for a cure, Ashitaka arrives at Tatara, the Iron Town, where he finds himself embroiled in a fierce conflict: Lady Eboshi of Tatara, promoting constant deforestation, stands against Princess San and the sacred spirits of the forest, who are furious at the destruction brought by the humans. As the opposing forces of nature and mankind begin to clash in a desperate struggle for survival, Ashitaka attempts to seek harmony between the two, all the while battling the latent demon inside of him. Princess Mononoke is a tale depicting the connection of technology and nature, while showing the path to harmony that could be achieved by mutual acceptance. [Written by MAL Rewrite]"
431,1 hr 59 min,Howl no Ugoku Shiro,G - All Ages,"That jumbled piece of architecture, that cacophony of hissing steam and creaking joints, with smoke billowing from it as it moves on its own... That castle is home to the magnificent wizard Howl, infamous for both his magical prowess and for being a womanizer—or so the rumor goes in Sophie Hatter's small town. Sophie, as the plain daughter of a hatmaker, does not expect much from her future and is content with working hard in the shop. However, Sophie's simple life takes a turn for th